In [26]:
# You may need to restart your runtime prior to this, to let your installation take effect
# Some basic setup
# Setup detectron2 logger
import detectron2
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import cv2
import random
import torch
import matplotlib.pyplot as plt
import expandMask
# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.utils.visualizer import VisImage
from detectron2.data import MetadataCatalog

In [27]:
im = cv2.imread("/home/asung/detImage/h.jpg")
print(im.shape)

(1080, 1440, 3)


In [28]:
cfg = get_cfg()
# add project-specific config (e.g., TensorMask) here if you're not running a model in detectron2's core library
# start path in configs [dir]
fileName = "COCO-Keypoints/keypoint_rcnn_R_50_FPN_3x.yaml"
model = model_zoo.get_config_file(fileName)

cfg.merge_from_file(model)
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.5  # set threshold for this model
cfg.merge_from_list(['MODEL.DEVICE','cpu'])

# Find a model from detectron2's model zoo. You can either use the https://dl.fbaipublicfiles.... url, or use the detectron2:// shorthand
cfg.MODEL.WEIGHTS = "detectron2://"+model_zoo.get_weight_suffix(fileName)
predictor = DefaultPredictor(cfg)
outputs = predictor(im)

In [29]:
# look at the outputs. See https://detectron2.readthedocs.io/tutorials/models.html#model-output-format for specification
#print(outputs)
#print("\n\n\n\n\n\n")
print(outputs["instances"],"\n\n")
print("class: ",outputs["instances"].pred_classes.shape,"\n",outputs["instances"].pred_classes,"\n")
print("boxes: ",outputs["instances"].pred_boxes.tensor.shape,"\n",outputs["instances"].pred_boxes,"\n")
print("scores: " ,outputs["instances"].scores.shape ,"\n",outputs["instances"].scores,"\n")
print("keypoint:\n",outputs["instances"].pred_keypoints.shape ,"\n",outputs["instances"].pred_keypoints,"\n")

Instances(num_instances=25, image_height=1080, image_width=1440, fields=[pred_boxes, scores, pred_classes, pred_keypoints]) 


class:  torch.Size([25]) 
 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0]) 

boxes:  torch.Size([25, 4]) 
 Boxes(tensor([[6.2089e+02, 7.1486e+02, 8.8054e+02, 1.0758e+03],
        [8.4417e+02, 6.5449e+02, 1.0264e+03, 9.1244e+02],
        [6.3818e+01, 6.5377e+02, 2.7729e+02, 1.0221e+03],
        [1.0156e+03, 1.0021e+03, 1.2124e+03, 1.0781e+03],
        [1.0238e+00, 6.5231e+02, 9.1711e+01, 8.2163e+02],
        [3.2150e+02, 6.4347e+02, 5.1516e+02, 9.1342e+02],
        [1.2064e+03, 6.1525e+02, 1.3207e+03, 7.3546e+02],
        [1.1963e+03, 6.5313e+02, 1.3286e+03, 8.0556e+02],
        [2.1894e+02, 6.3339e+02, 3.3521e+02, 7.4637e+02],
        [9.6548e+02, 6.3944e+02, 1.1059e+03, 7.9714e+02],
        [2.9315e+02, 8.6108e+02, 6.0347e+02, 1.0769e+03],
        [1.0600e+03, 6.8772e+02, 1.2622e+03, 9.1019e+02],
        [6.0587e+02,

In [44]:
outputs["instances"].pred_keypoints[24][2][2]

tensor(0.0879)

In [15]:
poslist = []
boxes = outputs["instances"].pred_boxes.tensor
pred = outputs['instances'].pred_classes
masks = outputs["instances"].pred_masks
for i in range(outputs['instances'].scores.shape[0]):
    if pred[i] == 0: 
        wide = abs(boxes[i][2] - boxes[i][0])*abs(boxes[i][3]-boxes[i][1])
        poslist.append(int(wide))
    else:
        poslist.append(0)

print(poslist)
idx = poslist.index(max(poslist))

x_s,y_s,x_d,y_d = boxes[idx]

[1904, 1783, 1815, 1687, 1683, 1817, 1830, 1591, 1419, 2023, 1166, 3966, 1992, 1908, 1661, 1887, 1629, 1799]


In [16]:

# We can use `Visualizer` to draw the predictions on the image.
v = Visualizer(im[:, :, ::-1], MetadataCatalog.get(cfg.DATASETS.TRAIN[0]), scale=1.0)
v = v.draw_instance_predictions(outputs["instances"].to("cpu"))
print(type(v))
#                   box(x_s,y_s,x_d,y_d)
#                   img[y_s:y_d, x_s:x_d]
vtmp = v.get_image()[:,:,::-1]#[244:480,126:459,::-1]
print(type(vtmp))

tmp = im[int(y_s):int(y_d),int(x_s):int(x_d)] #네모 박스 칸만큼 slie


tim = torch.from_numpy(im.copy())  #tim은  im의 복사본, tensor로 변환

be = torch.where(masks[idx]!=True)
    
#remove background
tim[be] = 224

tim = tim.numpy()
print(be)
#tim = tim[int(y_s):int(y_d),int(x_s):int(x_d)]
print(tim[be])
def fitsize(im,y_s,y_d,x_s,x_d):
    y_s,y_d,x_s,x_d = int(y_s),int(y_d),int(x_s),int(x_d)
    return im[y_s:y_d,x_s:x_d]

<class 'detectron2.utils.visualizer.VisImage'>
<class 'numpy.ndarray'>
(tensor([  0,   0,   0,  ..., 225, 225, 225]), tensor([  0,   1,   2,  ..., 220, 221, 222]))
[[224 224 224]
 [224 224 224]
 [224 224 224]
 ...
 [224 224 224]
 [224 224 224]
 [224 224 224]]


In [17]:
etim = torch.from_numpy(im.copy())
emask = expandMask.expand(masks[idx],20,y_s,y_d,x_s,x_d)
ebe = torch.where(emask!=True)
etim[ebe] = 224
etim = etim.numpy()
etim.shape

(226, 223, 3)

In [21]:
gauss = cv2.GaussianBlur(im,(5,5),1e+10)
median = cv2.medianBlur(im,5)
tr = torch.where(masks[idx] == True)
gauss[tr] = im[tr]
median[tr] = im[tr]
gauss = fitsize(gauss,int(y_s),int(y_d),int(x_s),int(x_d))
median = fitsize(median,int(y_s),int(y_d),int(x_s),int(x_d))

# 배경자르기..
cv2.imshow('be',tim)
cv2.imshow('ori',tmp)
cv2.imshow('imagin',gauss)
cv2.imshow('median',median)
cv2.imshow('test',etim)
cv2.waitKey(0)
cv2.destroyAllWindows()